In [65]:
import pandas as pd
import numpy as np

from collections import Counter
from sklearn_pandas import CategoricalImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from impyute.imputation.cs import fast_knn
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from impyute.imputation.cs import mice
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

In [2]:
data = pd.read_pickle("files/Optima_Data_Report_Cases_9584_filled_pickle")

In [3]:
X = ['HISTORY PATIENT 74-119: (076) HISTORY OF STROKE',
                  'MEDICAL ASSESSMENT V 2010: STROKE',
                  'HACHINSKI ISCHAEMIC: HISTORY OF STROKE']

In [4]:
y = ['OPTIMA DIAGNOSES V 2010: CERBRO-VASCULAR DISEASE PRESENT']

In [5]:
columns = X + y

In [6]:
subset = data[columns]

In [7]:
subset = subset[subset[y[0]].notna()]

In [8]:
subset = subset[subset[y[0]].isin([8, 9])!=True]

In [9]:
subset[X].isna().sum().sum() / (subset[X].shape[0] * subset[X].shape[1])

0.7054278416347382

In [10]:
subset = subset.fillna(-1)

In [11]:
subset = subset.reset_index(drop=True)

In [12]:
train, test = train_test_split(subset, test_size=0.33, random_state=42, shuffle=True, stratify=subset[y])

In [13]:
y_test = test[y[0]].copy().values

In [14]:
test[y[0]] = np.nan

/home/william/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [71]:
subset.shape

(5220, 4)

In [72]:
Counter(subset[y].values.reshape(-1))

Counter({0.0: 3897, 1.0: 716, 2.0: 269, 4.0: 301, 3.0: 37})

In [15]:
imputer_categorical = CategoricalImputer()

In [16]:
imputer_categorical.fit(train.values)

CategoricalImputer(copy=True, fill_value='?', missing_values='NaN',
                   strategy='most_frequent')

In [17]:
imputer_categorical_y = imputer_categorical.transform(test.values)[:, -1]

In [18]:
accuracy_score(y_test, imputer_categorical_y)

0.0

In [19]:
imputer_simple_median = SimpleImputer( strategy='median') 

In [20]:
imputer_simple_median.fit(train.values)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [21]:
imputer_simple_median_y = imputer_simple_median.transform(test.values)[:, -1]

In [22]:
accuracy_score(y_test, imputer_simple_median_y)

0.7463726059199072

In [23]:
imputer_simple_most_frequent = SimpleImputer( strategy='most_frequent') 

In [24]:
imputer_simple_most_frequent.fit(train.values)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='most_frequent', verbose=0)

In [25]:
imputer_simple_most_frequent_y = imputer_simple_most_frequent.transform(test.values)[:, -1]

In [26]:
accuracy_score(y_test, imputer_simple_most_frequent_y)

0.7463726059199072

In [27]:
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [28]:
mlp.fit(train[X].values, train[y].values)

/home/william/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [29]:
mlp_y = mlp.predict(test[X].values)

In [30]:
accuracy_score(y_test, mlp_y)

0.7974463145676146

In [31]:
rf = RandomForestClassifier(max_depth=2, random_state=0)

In [32]:
rf.fit(train[X].values, train[y].values)

/home/william/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/william/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [33]:
rf_y = rf.predict(test[X].values)

In [34]:
accuracy_score(y_test, rf_y)

0.7968659315147998

In [35]:
lor = LogisticRegression(random_state=0).fit(train[X].values, train[y].values)

/home/william/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/william/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/william/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [36]:
lor_y = lor.predict(test[X].values)

In [37]:
accuracy_score(y_test, lor_y)

0.7452118398142774

In [38]:
np.unique(y_test)/4

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [39]:
1/5

0.2

In [40]:
train[y] = train[y]/4

/home/william/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [41]:
lir = LinearRegression().fit(train[X].values, train[y].values)

In [42]:
lir_y = lir.predict(test[X].values)

In [43]:
max(lir_y)

array([0.29694827])

In [44]:
lir_y = lir_y.reshape(-1)

In [45]:
lir_y[lir_y<0.2] = 5
lir_y[lir_y<0.4] = 6
lir_y[lir_y<0.6] = 7
lir_y[lir_y<0.8] = 8
lir_y[lir_y<1.0] = 9
lir_y -= 5

In [46]:
accuracy_score(y_test, lir_y)

0.7446314567614626

In [47]:
Counter(y_test)

Counter({0.0: 1286, 4.0: 100, 1.0: 236, 2.0: 89, 3.0: 12})

In [48]:
Counter(train[y].values.reshape(-1))

Counter({0.0: 2611, 0.5: 180, 1.0: 201, 0.25: 480, 0.75: 25})

In [49]:
imputer_simple_mean = SimpleImputer( strategy='mean') 

In [50]:
imputer_simple_mean.fit(train.values)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

In [51]:
imputer_simple_mean_y = imputer_simple_mean.transform(test.values)[:, -1]

In [52]:
imputer_simple_mean_y[imputer_simple_mean_y<0.2] = 5
imputer_simple_mean_y[imputer_simple_mean_y<0.4] = 6
imputer_simple_mean_y[imputer_simple_mean_y<0.6] = 7
imputer_simple_mean_y[imputer_simple_mean_y<0.8] = 8
imputer_simple_mean_y[imputer_simple_mean_y<1.0] = 9
imputer_simple_mean_y -= 5

In [53]:
accuracy_score(y_test, imputer_simple_mean_y)

0.7463726059199072

In [54]:
subset_concat = pd.concat([train, test])

In [55]:
sys.setrecursionlimit(100000)

In [56]:
fast_knn_y = fast_knn(subset_concat.values, k=30)

/home/william/anaconda3/lib/python3.7/site-packages/impyute/imputation/cs/fast_knn.py:113: RuntimeWarning: invalid value encountered in true_divide
  weights = distances/np.sum(distances)


In [57]:
fast_knn_y = fast_knn_y[train.shape[0]:, -1]

In [58]:
fast_knn_y[np.isnan(fast_knn_y)] = -1

In [59]:
fast_knn_y[fast_knn_y<0.2] = 5
fast_knn_y[fast_knn_y<0.4] = 6
fast_knn_y[fast_knn_y<0.6] = 7
fast_knn_y[fast_knn_y<0.8] = 8
fast_knn_y[fast_knn_y<1.0] = 9
fast_knn_y -= 5

In [60]:
accuracy_score(y_test, fast_knn_y)

0.7556587347649448

In [61]:
mice_y = mice(subset_concat.values)

In [62]:
mice_y = mice_y[train.shape[0]:, -1]

In [63]:
mice_y[mice_y<0.2] = 5
mice_y[mice_y<0.4] = 6
mice_y[mice_y<0.6] = 7
mice_y[mice_y<0.8] = 8
mice_y[mice_y<1.0] = 9
mice_y -= 5

In [64]:
accuracy_score(y_test, mice_y)

0.7446314567614626

In [66]:
imputer_iterative = IterativeImputer(random_state=0)

In [67]:
imputer_iterative.fit(train.values)

IterativeImputer(add_indicator=False, estimator=None,
                 imputation_order='ascending', initial_strategy='mean',
                 max_iter=10, max_value=None, min_value=None,
                 missing_values=nan, n_nearest_features=None, random_state=0,
                 sample_posterior=False, tol=0.001, verbose=0)

In [68]:
imputer_iterative_y = imputer_iterative.transform(test.values)[:, -1]

In [69]:
imputer_iterative_y[imputer_iterative_y<0.2] = 5
imputer_iterative_y[imputer_iterative_y<0.4] = 6
imputer_iterative_y[imputer_iterative_y<0.6] = 7
imputer_iterative_y[imputer_iterative_y<0.8] = 8
imputer_iterative_y[imputer_iterative_y<1.0] = 9
imputer_iterative_y -= 5

In [70]:
accuracy_score(y_test, imputer_iterative_y)

0.7463726059199072